# Tweet filtering

In [3]:
import pandas as pd
import numpy as np
import preprocessor as p
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from functools import reduce
import os

In [ ]:
PATH_DIR = './'
os.chdir(PATH_DIR)

In [2]:
PATH = 'Data/Dataset 2022/'
PATH_FILE_XLSX = 'tweets.xlsx'

df_tweets = pd.read_excel(PATH+PATH_FILE_XLSX)

In [3]:
df_tweets.head()

,tweet_id,user_id,text,created_at,in_reply_to_status_id,in_reply_to_user_id,quote_count,reply_count,retweet_count,favorite_count,entities,type
0,1593016536024769024,1529463808053616896,"This place is pretty dead... By which I mean, ...",Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",tweet
1,1593016536016749056,1553398174584377088,@Mark_Baden And this is just the beginning...🥶😳,Wed Nov 16 23:01:53 +0000 2022,1.592998e+18,474414285.0,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",tweet
2,1593016536016429056,1465584077751397888,RT @CantStopLion: 📺🔊 Magoon Devin Nunes is the...,Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",RT
3,1593006457330761984,1018632078269616000,📺🔊 Magoon Devin Nunes is the face of the Derp ...,Wed Nov 16 22:21:50 +0000 2022,NaN,NaN,0,0,2,2,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",tweet
4,1593016536016749056,100084136,RT @youhearbiggirls: How do I sum up the best ...,Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",RT


In [4]:
OG_tweets = df_tweets[df_tweets['type'] != 'RT']

In [5]:
def find_hashtags(text):
    return re.findall('(#+[a-zA-Z0-9(_)]{2,})', text)

def get_entities(text):
    parsed_tweet = p.parse(text)
    chosen_entities = [parsed_tweet.hashtags, parsed_tweet.emojis, parsed_tweet.smileys, parsed_tweet.numbers, parsed_tweet.urls]
    chosen_names = ['hashtags', 'emojis', 'smileys', 'numbers', 'urls']
    dict_ent = {}

    for entity,name in zip(chosen_entities, chosen_names):
        dict_ent[name] = []
        if (entity != None):
            for i in range(len(entity)):
                dict_ent[name].append(entity[i].match)

    return dict_ent

def remove_diacritics(text):
    tweet = re.sub("[àáâãäå]","a", text)
    tweet = re.sub("[èéêë]","e", tweet)
    tweet = re.sub("[ìíîï]","i", tweet)
    tweet = re.sub("[òóôö]","o", tweet)
    tweet = re.sub("[ùúûü]","u", tweet)
    return tweet

contractions = { 
"ain't": "is not",
#"aren't": "are not",
"can't": "can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
#"couldn't": "could not",
"couldn't've": "could not have",
#"didn't": "did not",
#"doesn't": "does not",
#"don't": "do not",
#"hadn't": "had not",
"hadn't've": "had not have",
#"hasn't": "has not",
#"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
#"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
#"mightn't": "might not",
"mightn't've": "might not have",
#"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
#"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
#"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
#"she's": "she is",
#"should've": "should have",
#"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
#"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
#"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
#"won't": "will not",
"won't've": "will not have",
"would've": "would have",
#"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
#"you'd": "you would",
"you'd've": "you would have",
#"you'll": "you will",
"you'll've": "you will have",
#"you're": "you are",
#"you've": "you have"
"u": "you",
"u're": "you are",
"ure": "you are",
"u r": "you are",
"ur": "your"
}

def expand_contractions(text):
    for key,val in contractions.items():
        text = re.sub("\\b"+key+"\\b", val, text)
    return text

StopWords = stopwords.words("english")

def clean_tweet(text):
    tweet = re.sub('\n', ' ', text) # remove line jumps
    tweet = re.sub("\u2026", " ", tweet) # remove triple dots
    tweet = p.clean(tweet)
    tweet = re.sub("[0-9]", " ", tweet) # remove numbers
    tweet = re.sub('\&amp;', 'and', tweet) # replace ampersands
    tweet = re.sub('['+punctuation + ']+', ' ', tweet) # remove punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', tweet) # remove repeated characters
    tweet = tweet.lower() # make lower case
    tweet = remove_diacritics(tweet)
    tweet = expand_contractions(tweet)
    tweet = ' '.join([word for word in tweet.split() if word not in StopWords])
    return tweet

def stem_tweet(text):
    ps = PorterStemmer()
    words = word_tokenize(text)
    return reduce(lambda x, y: x + " " + ps.stem(y), words, "")

In [6]:
filtered_text = pd.Series(dtype=str)
tokenized_text = pd.Series(dtype=str)
stemmed_text = pd.Series(dtype=str)
hashtags = pd.Series()
emojis = pd.Series()
smileys = pd.Series()
numbers = pd.Series()
urls = pd.Series()
punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

for idx,text in OG_tweets['text'].items():
    try:
        tweet = clean_tweet(text)

        filtered_text[idx] = tweet
        stemmed_text[idx] = stem_tweet(tweet)

        tokenized_text[idx] = p.tokenize(text)

        entities = get_entities(text)

        hashtags[idx] = entities['hashtags']
        emojis[idx] = entities['emojis']
        smileys[idx] = entities['smileys']
        numbers[idx] = entities['numbers']
        urls[idx] = entities['urls']
    except:
        print("Error in tweet",idx)
        print("Text: ",text)

Error in tweet 4649
Text:  0


In [7]:
#df_tweets = df_tweets.drop(['filtered_text', 'tokenized_text', 'stemmed_text', 'hashtags', 'emojis', 'smileys', 'numbers', 'urls'], axis=1)

df_tweets['filtered_text'] = filtered_text
df_tweets['tokenized_text'] = tokenized_text
df_tweets['stemmed_text'] = stemmed_text
df_tweets['hashtags'] = hashtags
df_tweets['emojis'] = emojis
df_tweets['smileys'] = smileys
df_tweets['numbers'] = numbers
df_tweets['urls'] = urls

df_tweets = df_tweets.fillna({'hashtags':'[]', 'emojis':'[]', 'smileys':'[]', 'numbers':'[]', 'urls':'[]'}).apply(list)

In [8]:
df_tweets.head()

,tweet_id,user_id,text,created_at,in_reply_to_status_id,in_reply_to_user_id,quote_count,reply_count,retweet_count,favorite_count,entities,type,filtered_text,tokenized_text,stemmed_text,hashtags,emojis,smileys,numbers,urls
0,1593016536024769024,1529463808053616896,"This place is pretty dead... By which I mean, ...",Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",tweet,place pretty dead mean nothing kill,"This place is pretty dead... By which I mean, ...",place pretti dead mean noth kill,[],[],[],[],[]
1,1593016536016749056,1553398174584377088,@Mark_Baden And this is just the beginning...🥶😳,Wed Nov 16 23:01:53 +0000 2022,1.592998e+18,474414285.0,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",tweet,beginning,$MENTION$ And this is just the beginning...$EM...,begin,[],[😳],[],[],[]
2,1593016536016429056,1465584077751397888,RT @CantStopLion: 📺🔊 Magoon Devin Nunes is the...,Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",RT,NaN,NaN,NaN,[],[],[],[],[]
3,1593006457330761984,1018632078269616000,📺🔊 Magoon Devin Nunes is the face of the Derp ...,Wed Nov 16 22:21:50 +0000 2022,NaN,NaN,0,0,2,2,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",tweet,magoon devin nunes face derp state,$EMOJI$$EMOJI$ Magoon Devin Nunes is the face ...,magoon devin nune face derp state,[],"[📺, 🔊]",[],[],[https://t.co/8lQozzZJxH]
4,1593016536016749056,100084136,RT @youhearbiggirls: How do I sum up the best ...,Wed Nov 16 23:01:53 +0000 2022,NaN,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",RT,NaN,NaN,NaN,[],[],[],[],[]


In [9]:
PATH = 'Data/Dataset 2022/'
FILE_NAME = 'filtered_tweets.xlsx'

df_tweets.to_excel(PATH+FILE_NAME+'.xlsx', engine='xlsxwriter')